In [ ]:
import cv2
from PIL import Image
import numpy as np

In [ ]:
image = cv2.imread('/Users/shravanprasanth/Documents/AIScouter/runs/detect/predict/crops/Robot/test5.jpg')

height, width, _ = image.shape
start_row = height // 2

bottom_half_img = image[start_row:height, :]
cv2.imwrite('bottom_half.jpg', bottom_half_img)
cv2.imshow('Bottom Half', bottom_half_img)

In [ ]:
average_rgb = np.mean(bottom_half_img, axis=(0, 1))

print('Average RGB:', average_rgb)

In [ ]:
!pip3 install levenshtein
!pip3 install easyocr

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO

# Load the YOLOv8 model (ensure your model is correctly trained for detecting robots)
model = YOLO('/Users/shravanprasanth/Documents/AIScouter/src/models/y8v7.pt')  # Replace with your custom model if applicable

# Open the input video
input_video_path = '/Users/shravanprasanth/Documents/AIScouter/src/videos/dcmp58.mp4'
cap = cv2.VideoCapture(input_video_path)

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/Users/shravanprasanth/Documents/AIScouter/src/videos/output/dcmp58-colors.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO detection on the frame
    results = model(frame)
    
    for result in results:
        boxes = result.boxes  # Get detected boxes

        for box in boxes:
            # Extract the bounding box coordinates
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            # Crop the bottom half of the bounding box
            start_row = (y1 + y2) // 2
            bottom_half = frame[start_row:y2, x1:x2]

            # Calculate the average RGB values
            average_rgb = np.mean(bottom_half, axis=(0, 1))

            # Check the RGB condition for labeling
            label = 'Blue' if average_rgb[0] > average_rgb[-1] else 'Red'
            color = (255, 0, 0) if label == 'Blue' else (0, 0, 255)

            # Draw the bounding box and label on the frame
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Write the processed frame to the output video
    out.write(frame)

# Release the video objects
cap.release()
out.release()

print("Processing complete. Output saved to", output_video_path)


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import easyocr
from Levenshtein import distance as levenshtein_distance

# Load the YOLOv8 model (ensure your model is correctly trained for detecting robots)
model = YOLO('/Users/shravanprasanth/Documents/AIScouter/src/models/y8v7.pt')

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Define the team numbers for red and blue teams
red_team_numbers = ["75", "2722", "1391"]  # Replace with actual red team numbers
blue_team_numbers = ["56", "5401", "8513"]  # Replace with actual blue team numbers

# Open the input video
input_video_path = '/Users/shravanprasanth/Documents/AIScouter/src/videos/dcmp58.mp4'
cap = cv2.VideoCapture(input_video_path)

# Get video properties
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

# Define the codec and create a VideoWriter object to save the output video
output_video_path = '/Users/shravanprasanth/Documents/AIScouter/src/videos/output/dcmp58-leven.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

# Process each frame
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    # Run YOLO detection on the frame
    results = model(frame)
    
    for result in results:
        boxes = result.boxes  # Get detected boxes

        for box in boxes:
            # Extract the bounding box coordinates
            x1, y1, x2, y2 = map(int, box.xyxy[0])

            # Crop the bottom half of the bounding box
            start_row = (y1 + y2) // 2
            bottom_half = frame[start_row:y2, x1:x2]

            # Calculate the average RGB values
            average_rgb = np.mean(bottom_half, axis=(0, 1))

            # Determine the team color based on RGB values
            label = 'Blue' if average_rgb[0] > average_rgb[-1] else 'Red'
            color = (255, 0, 0) if label == 'Blue' else (0, 0, 255)

            # OCR: Recognize text in the bottom half of the box
            ocr_result = reader.readtext(bottom_half)
            detected_team_number = None

            if ocr_result:
                # Get the detected text
                detected_text = ocr_result[0][-2].replace(" ", "")  # Extract and clean text

                # Find the closest matching team number using Levenshtein distance
                if label == 'Red':
                    closest_match = min(red_team_numbers, key=lambda num: levenshtein_distance(detected_text, num))
                else:
                    closest_match = min(blue_team_numbers, key=lambda num: levenshtein_distance(detected_text, num))

                detected_team_number = closest_match

            # Draw the bounding box and label on the frame
            if detected_team_number:
                label += f" {detected_team_number}"
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

    # Write the processed frame to the output video
    out.write(frame)

# Release the video objects
cap.release()
out.release()

print("Processing complete. Output saved to", output_video_path)


In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image
import easyocr
from Levenshtein import distance as levenshtein_distance
from difflib import SequenceMatcher
from rapidfuzz.distance import JaroWinkler

# Load the YOLOv8 model (ensure your model is correctly trained for detecting robots)
model = YOLO('/Users/shravanp/Coding/Robotics/AIScouter/src/models/y8v7.pt')

# Initialize EasyOCR reader
reader = easyocr.Reader(['en'])

# Define the team numbers for red and blue teams
red_team_numbers = ["75", "2722", "1391"]  # Replace with actual red team numbers
blue_team_numbers = ["56", "5401", "8513"]  # Replace with actual blue team numbers

# Provide the path to the input image
input_image_path = '/Users/shravanp/Coding/Robotics/AIScouter/src/images/test.png'
output_image_path = '/Users/shravanp/Coding/Robotics/AIScouter/src/images/output/test.png'

# Read the image
image = cv2.imread(input_image_path)

# Run YOLO detection on the image
results = model(image)
x = None
for result in results:
    boxes = result.boxes  # Get detected boxes

    for box in boxes:
        # Extract the bounding box coordinates
        x1, y1, x2, y2 = map(int, box.xyxy[0])

        # Crop the bottom half of the bounding box
        start_row = (y1 + y2) // 2
        bottom_half = image[start_row:y2, x1:x2]

        gray = cv2.cvtColor(bottom_half, cv2.COLOR_BGR2GRAY)

        # Apply GaussianBlur to reduce noise and improve OCR accuracy

        # # Apply thresholding to binarize the image
        # # _, thresh = cv2.threshold(blurred, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

        # # Apply morphological operations (optional, depends on image quality)
        # # kernel = np.ones((2, 2), np.uint8)
        # # morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)

        clahe = cv2.createCLAHE(clipLimit=1, tileGridSize=(10, 10))
        enhanced_gray = clahe.apply(gray)
        upscale_model = cv2.dnn_superres.DnnSuperResImpl_create()
        upscale_model.readModel('/Users/shravanp/Coding/Robotics/AIScouter/src/models/ESPCN_x4.pb')  # Path to the pre-trained model
        upscale_model.setModel('espcn', 4)  # 'ESPCN' model with scale factor 4

        # Upscale the image
        upscaled_image = upscale_model.upsample(enhanced_gray)

        # Sharpen the image using an unsharp mask
        # gaussian_blur = cv2.GaussianBlur(enhanced_gray, (9, 9), 10.0)

        # Calculate the average RGB values
        average_rgb = np.mean(bottom_half, axis=(0, 1))

        # Determine the team color based on RGB values
        label = 'Blue' if average_rgb[0] > average_rgb[-1] else 'Red'
        color = (255, 0, 0) if label == 'Blue' else (0, 0, 255)

        # OCR: Recognize text in the bottom half of the box
        # upscaled_img = cv2.resize(upscaled_image, None, fx=2, fy=2, interpolation=cv2.INTER_CUBIC)



        assigned_teams = {}

            if detected_text:
                # Calculate Levenshtein distances and find the closest match
                distances = {num: levenshtein_distance(detected_text, num) for num in team_numbers}
                closest_match, min_distance = min(distances.items(), key=lambda item: item[1], default=(None, None))

                if min_distance is not None:  # No threshold, just check if a match was found
                    detected_team_number = closest_match
                    team_numbers.remove(detected_team_number)
                else:
                    detected_team_number = "Unknown"
            else:
                detected_team_number = "Unknown"

            # Update the assignment dictionary
            assigned_teams[(x1, y1, x2, y2)] = detected_team_number

            # Draw the bounding box and label on the image
            cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
            cv2.putText(frame, f"{detected_team_number}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

        # Handle any remaining team numbers
        for box in boxes:
            x1, y1, x2, y2, _, _ = box
            if assigned_teams.get((x1, y1, x2, y2)) == "Unknown":
                if team_numbers:
                    # Assign the next available team number
                    detected_team_number = team_numbers.pop(0)
                else:
                    # Handle cases where no team numbers are available
                    detected_team_number = "Unknown"

                # Update the assignment dictionary and draw on the image
                assigned_teams[(x1, y1, x2, y2)] = detected_team_number
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{detected_team_number}", (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, color, 2)

# Save the processed image
cv2.imwrite(output_image_path, image)

In [ ]:
# 1. Normalization
def normalize_image(img):
    norm_img = np.zeros((img.shape[0], img.shape[1]), dtype=np.float32)
    normalized_img = cv2.normalize(img, norm_img, 0, 255, cv2.NORM_MINMAX)
    return np.uint8(normalized_img)


# 3. Image Scaling
def set_image_dpi(file_path):
    im = Image.open(file_path)
    length_x, width_y = im.size
    factor = min(1, float(1024.0 / length_x))
    size = int(factor * length_x), int(factor * width_y)
    im_resized = im.resize(size, Image.ANTIALIAS)
    temp_file = tempfile.NamedTemporaryFile(delete=False, suffix='.png')
    temp_filename = temp_file.name
    im_resized.save(temp_filename, dpi=(300, 300))
    return temp_filename

# 4. Noise Removal
def remove_noise(image):
    return cv2.fastNlMeansDenoisingColored(image, None, 10, 10, 7, 15)

# 5. Thinning and Skeletonization
def thin_image(image):
    kernel = np.ones((5, 5), np.uint8)
    return cv2.erode(image, kernel, iterations=1)

# 6. Grayscale Conversion
def get_grayscale(image):
    return cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

# # 7. Thresholding or Binarization
# def thresholding(image):
#     return cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

In [ ]:
!pip3 install opencv-contrib-python

In [ ]:
import cv2
import numpy as np
from PIL import Image
import easyocr
reader = easyocr.Reader(['en'])

image_path = "/Users/shravanp/Coding/Robotics/AIScouter/src/images/75.png"
image = cv2.imread(image_path)

upscale_model = cv2.dnn_superres.DnnSuperResImpl_create()
upscale_model.readModel('/Users/shravanp/Coding/Robotics/AIScouter/src/models/ESPCN_x4.pb')  # Path to the pre-trained model
upscale_model.setModel('espcn', 4)  # 'ESPCN' model with scale factor 4

# Upscale the image
upscaled_image = upscale_model.upsample(image)
print(reader.readtext(upscaled_image, allowlist="0123456789", contrast_ths=0.1))

In [ ]:
red_team_numbers = ["75", "2722", "1391"] 
blue_team_numbers = ["56", "5401", "8513"]
closest_match = max(red_team_numbers, key=lambda num: JaroWinkler.similarity("C", num))
closest_match

In [ ]:
import ncnn
import numpy as np
import cv2
from PIL import Image

In [ ]:
# Initialize NCNN with Vulkan backend (usually Vulkan is enabled by default if supported)
net = ncnn.Net()
net.opt.use_vulkan_compute = False  # Enable Vulkan

# Load the model
net.load_param("/Users/shravanp/Coding/Robotics/AIScouter/src/models/models-DF2K/x4.param")
net.load_model("/Users/shravanp/Coding/Robotics/AIScouter/src/models/models-DF2K/x4.bin")

In [ ]:
# Read input image
img = cv2.imread("/Users/shravanp/Coding/Robotics/AIScouter/src/images/1391.png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
h, w, c = img.shape

# Convert image to NCNN Mat
input_img = ncnn.Mat.from_pixels(img, ncnn.Mat.PixelType.PIXEL_BGR, w, h)

# Create NCNN extractor
ex = net.create_extractor()
ex.input("data", input_img)

# Run inference
output = ncnn.Mat()
ex.extract("output", output)

output_img = np.zeros((output.h, output.w, 3), dtype=np.uint8)

# Convert NCNN Mat to numpy array by accessing each pixel channel
for y in range(output.h):
    for x in range(output.w):
        output_img[y, x, 0] = output.channel(0)[y * output.w + x]
        output_img[y, x, 1] = output.channel(1)[y * output.w + x]
        output_img[y, x, 2] = output.channel(2)[y * output.w + x]

# Save the output image
output_img = cv2.cvtColor(output_img, cv2.COLOR_RGB2BGR)
Image.fromarray(np.array(output_img))

In [ ]:
levenshtein_distance("1301", ["1391", "75", "2722"])